# Collabarative filtering based Recommender System in Python Using Amazon Dataset


A recommender system is a subclass of information filtering system that seeks to predict the "rating" or "preference" a user would give to an item.

There are two variations of this system: content based and collaborative filtering based approach.

**Collaborative filtering** is a method of making automatic predictions about the interests of a user by collecting information regarding his preference by collecting data from many users (collaborating).

**Content-based filtering** methods work on a description of the item and a profile of the user's preferences.

# Data Collection

The dataset is available on Kaggle by the name of 'Amazon and Best Buy electronics'and we have linked it here with our code. It is a list of over 7,000 online reviews for 50 electronic products from websites like Amazon and Best Buy provided by Datafiniti's Product Database. The dataset includes the review date, source, rating, title, reviewer metadata, and more. It is a pre-crawled dataset, taken as subset of a bigger dataset which is availabe through Datafiniti.




# Importing the necessary libraries

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.sparse
import nltk
import os
import json
import math
import csv
import re
import time
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from sklearn.externals import joblib
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from collections import Counter
from surprise import KNNWithMeans
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD


# Accessing dataset
Here we are accessing dataset by given link of reading files in input directory. We are renaming the columns so that we can access those columns which are needed by their names.

In [ ]:
data_read=pd.read_csv("/kaggle/input/amazon-and-best-buy-electronics/DatafinitiElectronicsProductData.csv",delimiter=',', 
                           names = ['id1', 'asins1', 'brand1', 'categories1', 'colors1', 'dateAdded1'
                                    , 'dateUpdated1', 'dimensions1', 'ean1', 'imageURL1', 'keys1', 'manufacturer1'
                                    , 'manufacturerno1', 'name1', 'primarycategory1', 'reviewDate1', 'reviewDateSeen1'
                                    , 'reviewsDoRecom1','reviewsNumHelp1','reviewsRating1','reviewsSourceURL1',
                                   'reviewsText1','reviewsTitle1','reviewsUsername1','sourceURL1','upc1','weight1',])


# Displaying the first 5 rows of dataset

In [ ]:
data_read.head()

# Accessing only the needed columns in an order
The variable saves only these 4 columns in this particular order.

In [ ]:
print("Aceessing 4 columns of choice \n")
data4read = data_read[['reviewsUsername1', 'id1','reviewsRating1', 'reviewsText1']]
print("Done!")

# Displaying the first 5 rows of chosen columns

In [ ]:
data4read.head()

# Displaying the shape of chosen columns
The shape of data represents the 7300 rows of data and 4 different columns

In [ ]:
print("Shape of chosen columns:\n", data4read.shape)

# Displaying the datatypes of chosen columns

The datatype of chosen columns is Python object as shown below in the output cell.

In [ ]:
print("Type of chosen columns:\n",data4read.dtypes)

# Displaying the information of dataset
Here a concise summary of the dataset is displayed which shows that usernames and product IDs are unique and are 7300 in number while 164 ratings are not given and 5 reviwes have not been given.

In [ ]:
print('Information about data: \n')
data4read.info()

# Handling missing information
Here the missing values in columns is shown which as we had calculated previoulsy is  164 for ratings and 5 for reviews.

In [ ]:
missing_cols=data4read.isnull()
missing_cols=missing_cols.sum()
print('Number of missing values across columns: \n')
print(missing_cols)

# Analyzing distribution of ratings given to products

In [ ]:
with sns.axes_style('white'):
    g = sns.factorplot("reviewsRating1", data=data4read, aspect=2.0,kind='count')
    g.set_ylabels("Total number of ratings")

# Final data for use
Here we are printing count of unique items in each column

In [ ]:
print("The whole data: ")

un_reviews=np.unique(data4read.reviewsUsername1)
len_un_reviews=len(np.unique(data4read.reviewsUsername1))
print("Total number of Users:\n", len_un_reviews)
un_products=np.unique(data4read.id1)
len_un_products=len(un_products)
print("Total number of products:\n", len_un_products)
un_ratings=data4read.shape[0]
print("Total number of ratings:\n",un_ratings)

# Preprocessing on reviews
Here we are performing steps of text preprocessing. These steps are needed for transferring text from human language to machine-readable format for further processing. 


In [ ]:
reviews_read=data4read[['reviewsText1']]
print(reviews_read)

In [ ]:
#remove columns and now we have only one column 'reviews.txt'
import csv

input_file = '/kaggle/input/amazon-and-best-buy-electronics/DatafinitiElectronicsProductData.csv'
output_file = 'output.csv'
cols_to_remove = [0, 1, 2, 3, 4, 5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,22,23,24,25,26] # Column indexes to be removed (starts at 0)

cols_to_remove = sorted(cols_to_remove, reverse=True) # Reverse so we remove from the end first
row_count = 0 # Current amount of rows processed

with open(input_file, "r") as source:
    reader = csv.reader(source)
    with open(output_file, "w", newline='') as result:
        writer = csv.writer(result)
        for row in reader:
            row_count += 1
            print('\r{0}'.format(row_count), end='') # Print rows processed
            for col_index in cols_to_remove:
                del row[col_index]
            writer.writerow(row)

# Converting csv to txt 
Here we are converting csv to text file for ease of access during preprocessing

In [ ]:
output_file = 'output.csv'
csv_file = 'output.csv'
txt_file = 'reviews.txt'
with open(txt_file, "w") as my_output_file:
    with open(csv_file, "r") as my_input_file:
        [ my_output_file.write(" ".join(row)+'\n') for row in csv.reader(my_input_file)]
    my_output_file.close()

# Counting stopwords
Here we are counting number of stopwords in reviews text file.

In [ ]:
stop_words = set(stopwords.words('english'))
 
# Open and read in a text file.
with open('output.csv', "r") as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for lines in csv_reader: 
        txt_line = csv_file.read()
        txt_words = txt_line.split()

# stopwords found counter.
sw_found = 0
 
# If each word checked is not in stopwords list,
# then append word to a new text file.
for check_word in txt_words:
    if not check_word.lower() in stop_words:
        # Not found on stopword list, so append.
        appendFile = open('stopwords-removed.csv','a')
        appendFile.write(" "+check_word)
        appendFile.close()
    else:
        # It's on the stopword list
        sw_found +=1
        print(check_word)
 
print(sw_found,"stop words found and removed")
print("Saved as 'stopwords-removed.csv' ")


# POS tagging
Here we are doing POS tagging of reviews text file.

In [ ]:
with open('output.csv', "r") as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for lines in csv_reader: 
        txt_line = csv_file.read()
        #txt_words = txt_line.split()
        print("POS::")
        for x in txt_line:
            print(nltk.pos_tag(x))

# Stemming tokens
Here we are stemming tokens in reviews text file using the Porter Stemmer.

In [ ]:
d=0
porter = PorterStemmer()
with open('/kaggle/input/amazon-and-best-buy-electronics/DatafinitiElectronicsProductData.csv', "r") as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for lines in csv_reader: 
        txt_line = csv_file.read()
        txt_words = txt_line.split()
print("Porter Stemmer::")
for check_word in txt_words:
    print(check_word,porter.stem(check_word))

# Dropping the reviews column as it is of no use in collabarative filtering

In [ ]:
data4read.drop(['reviewsText1'], axis=1,inplace=True)

# Analysis of ratings given according to Username

In [ ]:
a=data4read.groupby(by='reviewsUsername1')['reviewsRating1'].count()
rated_products_per_user = a.sort_values(ascending=False)
rated_products_per_user.head()

# Describing the statistcal info of ratings data

In [ ]:
rated_products_per_user.describe()


# Analyzing recommendations based on popularity

In [ ]:
sum_rated_products_per_user=sum(rated_products_per_user >= 10)
print('No of rated product more than 10 per user : {}\n'.format(sum_rated_products_per_user) )


# Plotting itemID per their ratings
Here we are making a plot of itemID versus its ratings to get a deeper insight of the data. Here we are making a new dataframe which contains users who has given 20 or more ratings



In [ ]:
new_df=data4read.groupby("id1").filter(lambda x:x['reviewsRating1'].count() >=20)


In [ ]:
no_of_ratings_per_product = new_df.groupby(by='id1')['reviewsRating1'].count().sort_values(ascending=True)

fig = plt.figure(figsize=plt.figaspect(.25))
ax = plt.gca()
plt.plot(no_of_ratings_per_product.values)
plt.xlabel('Item')
plt.ylabel('No of ratings per item')
plt.title('Ratings per item')
plt.yticks(np.arange( 0.005))
ax.set_xticklabels([])

plt.show()

# Popular products
The number of ratings for a product are listed to understand the popular products as shown in the plot



In [ ]:
c=new_df.groupby('id1')['reviewsRating1'].count().sort_values(ascending=False)
c.head()

In [ ]:
pop_products = pd.DataFrame(new_df.groupby('id1')['reviewsRating1'].count())
most_popular = pop_products.sort_values('reviewsRating1', ascending=True)

most_popular.head(25).plot(kind = "barh", title="Popular items ratings")

# Collaberative filtering 

Collaborative filtering is commonly used for recommender systems. These techniques aim to fill in the missing entries of a user-item association matrix.



# Reading dataset
Getting the new dataframe which contains users who have given 10 or more ratings

In [ ]:
new_df=data4read.groupby("id1").filter(lambda x:x['reviewsRating1'].count() >=10)

In [ ]:
reader = Reader(rating_scale=(1, 5))
new_data = Dataset.load_from_df(new_df,reader)

# Splitting dataset 
Here we have splitted the dataset as such that the test size is 25% of data while training data size is 70% of data.

In [ ]:
train_set, test_set = train_test_split(new_data, test_size=0.25,random_state=10)

# Using collabarative filtering (Pearson similarity, item based)
Using k=5 for KNN with means and pearson based similarity to predict item-based collaborative filtering.


In [ ]:
algo4data = KNNWithMeans(k=5, sim_options={'name': 'pearson_baseline', 'user_based': False})
algo4data.fit(train_set)
test_prediction1 = algo4data.test(test_set)
test_prediction1

#  Using collabarative filtering (Cosine similarity, item based)

Using k=5 for KNN with means and cosine based similarity to predict item-based collaborative filtering.


In [ ]:
algo4data = KNNWithMeans(k=5, sim_options={'name': 'cosine', 'user_based': False})
algo4data.fit(train_set)
test_prediction2 = algo4data.test(test_set)
test_prediction2

# Using collabarative filtering (Pearson similarity, item based)

Using k=3 for KNN with means and pearson based similarity to predict item-based collaborative filtering.


In [ ]:
algo4data = KNNWithMeans(k=3, sim_options={'name': 'pearson_baseline', 'user_based': False})
algo4data.fit(train_set)
test_prediction3 = algo4data.test(test_set)
test_prediction3

# Making predictions for sparse matrices
Here after running the below script we get sparse matrix so that is solved by correlation matrix

In [ ]:
new_df1=new_df.head(1000)
matrix1 = new_df1.pivot_table(values='reviewsRating1', index='reviewsUsername1', columns='id1', fill_value=0, aggfunc='first')
matrix1.head()

# Transpose of matrix

In [ ]:
transposed_matrix1 = matrix1.T
transposed_matrix1.head()

# Decomposing the transposed matrix

In [ ]:
SVD = TruncatedSVD(n_components=8)
decomposed_matrix1 = SVD.fit_transform(transposed_matrix1)

# Making correlation matrix
Here correlation matrix is made out of the transposed matrix by using numpy library

In [ ]:
correlation_matrix1 = np.corrcoef(decomposed_matrix1)

In [ ]:
d=transposed_matrix1.index[5]

# Index no of item ID purchased by user

In [ ]:
iid = "AWIm0C3TYSSHbkXwx3S6"

product_names1 = list(transposed_matrix1.index)
pid1 = product_names1.index(iid)
print("Index no of item id purchased by user: \n")
print(pid1)

# Correlation for all items
Correlation for all items with the item purchased by this user based on items rated by other users who bought the same item

In [ ]:
correlation_pid = correlation_matrix1[pid1]
correlation_pid

# Recommending top 3 correlated products

In [ ]:
r=transposed_matrix1.index[correlation_pid > -0.20858196]
recommend1 = list(r)

recommend1.remove(iid) #removing itself from recommendations

recommend1[0:3]